Import packages

In [196]:
import numpy as np
import scipy as sp
import networkx as nx

Set file path

In [197]:
input_directory_path = 'neutral/PTH_polythiophene/coords/'
input_file_name = '16n_RuCap_PTH_opt.xyz'
base_file_name ='16n_RuCap_PTH_NICS_{}'

Open coordinates files and put the atoms coordinates into an array

In [198]:
# Open file
with open(input_directory_path + input_file_name) as f: # Insert file path (this one should work)   
    lines = f.readlines()

# Make list of file and cut first 2 lines (atom number)
coords = []
atoms = []
for line in lines[2:]:
    if line.strip():  # Skip empty lines
        atoms.append(line.split()[0])
        coords.append(line.split()[1:])


# Make array of list and convert to floats
output_array = []
for coord in coords:
    if coord:  # Skip empty elements
        output_array.append([float(coord[0]), float(coord[1]), float(coord[2])])
    else:
        break

# Convert coordinates to numpy array
all_coords = np.array(output_array)

Define function to calculate distances and create distance matrix of all the atoms in the molecule

In [199]:
# Define function to calculate distance between 2 points
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

# Get number of points needed in array
n_atoms = len(all_coords)

# Make empty distance matrix, n_atoms^2 in size
dist_matrix = np.zeros((n_atoms, n_atoms))

# Loop through each pair of points and calculate distance
for i in range(n_atoms):
    for j in range(i, n_atoms):
        dist_matrix[i,j] = distance(all_coords[i], all_coords[j])
        dist_matrix[j,i] = dist_matrix[i,j]  # Fill out matrix since distance matrices are symmetric

Specify bond length and convert the distance matrix into an adjacency matrix

In [200]:
bond_len = 2  # Bond length threshold
adj_matrix = np.zeros((n_atoms, n_atoms)) # Make empty adjacency matrix, n_atoms^2 in size
for i in range(n_atoms):
    for j in range(i+1, n_atoms):
        if dist_matrix[i,j] < bond_len:
            adj_matrix[i,j] = adj_matrix[j,i] = 1

Create a list of arrays of the atom numbers of all the cycles in the adjacency matrix and find planes of the cycles

In [201]:
# Produce a list of arrays of the atom numbers of the atoms in cycles in the molecule
cycles = [c for c in nx.cycle_basis(nx.from_numpy_array(adj_matrix)) if len(c)>=5]

In [202]:
# Produce a list of arrays of the atom numbers of the atoms in cycles in the molecule
cycles = [c for c in nx.cycle_basis(nx.from_numpy_array(adj_matrix)) if len(c)>=5]

# for c in cycles:
#     print(np.sort(np.array(c)+1))

def find_planes(cycles, all_coords):
    normals = []  ## MDP
    centroids = []

    for atoms in cycles:
        ring_coords = all_coords[atoms]
        centroid = np.average(ring_coords,axis=0)
        ring_coords = ring_coords-centroid
        u,s,v = np.linalg.svd(ring_coords)

        normal = v[2,:]

        normals.append(normal)
        centroids.append(centroid)

    return normals,centroids ## MDP

normals,centroids=find_planes(cycles, all_coords) ## MDP


In [203]:
rotated_coords = []

def rotate_molecule(all_coords, normals, centroids,centroidAtOrigin=False):
    z = np.array([0,0,1.])
    rotated_coords=[]
    for ii,nn in enumerate(normals):
        # from https://stackoverflow.com/questions/45142959/calculate-rotation-matrix-to-align-two-vectors-in-3d-space?noredirect=1&lq=1
        

        a, b = (nn / np.linalg.norm(nn)).reshape(3), (z / np.linalg.norm(z)).reshape(3)
        v = np.cross(a, b)
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        rotmat = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

        
        cc = all_coords - centroids[ii]

        if centroidAtOrigin:
            new_coords = np.dot(rotmat,cc.T).T
        else:
            new_coords = np.dot(rotmat,cc.T).T + centroids[ii]
        rotated_coords.append(new_coords)

    return rotated_coords

rotated_coords= rotate_molecule(all_coords, normals, centroids, centroidAtOrigin=True)


def makeXYZ(atoms,coords):
    xyz=[]
    for ii,cc in enumerate(coords):
        xyz.append(atoms[ii]+" {} {} {}".format(*cc))
    
    return '\n'.join(xyz)


for ii,coords in enumerate(rotated_coords):
    with open(input_directory_path + base_file_name.format(ii) + '.com', 'w') as f:
        f.writelines([
            "%chk={}.chk\n".format(base_file_name.format(ii)),
            "%nprocshared=8",'\n',
            "%mem=60GB",'\n', 
            "#p nmr=giao b1lyp def2svp iop(3/76=1000003500,3/77=0650006500,3/78=1000010000)",'\n\n', 
            "NICS calculations on neutral oligomers, Cycle: " + str(np.sort(np.array(cycles[ii])+1)), '\n\n', 
            "0 1", '\n'])
        f.writelines(makeXYZ(atoms,coords))
        f.writelines(["\n","Bq 0.0 0.0 1.0","\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n" ])
